<a href="https://colab.research.google.com/github/vaibhavjain2005/ResumeAiGenModel/blob/main/final_resume_api/api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install flask pyngrok transformers torch sentencepiece reportlab -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.7 MB/s eta 0:00:00


In [ ]:

import io
from flask import Flask, request, jsonify, send_file
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.enums import TA_LEFT, TA_CENTER
from reportlab.lib import colors
from datetime import datetime
import json

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
print("Loading FLAN-T5-Large model... (better quality, still free)")
model_path = "jain05vaibhav/flan-t5-fine-tuned-resume-stable"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
print("✅ Model loaded successfully!")

Loading FLAN-T5-Large model... (better quality, still free)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

✅ Model loaded successfully!


In [2]:
 #AI Functions for Resume Generation
def extract_keywords_from_job(job_description):
    """Extract important keywords from job description"""
    prompt = f"""Extract key skills and technologies from this job posting. List only the most important ones as comma-separated keywords.

Job Description: {job_description}

Important keywords:"""

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=100,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    keywords = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return keywords.strip()


def tailor_resume_experience(experience_text, keywords):
    """Rewrite experience bullet to match job requirements"""
    prompt = f"""Rewrite this work experience bullet point to be professional and impactful. Use action verbs and be specific.

Original: {experience_text}

Improved bullet point:"""

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=100,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    improved = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # If the output is too similar to input or just keywords, return improved original
    if len(improved) < 10 or improved == keywords:
        return experience_text
    return improved.strip()


def tailor_project_description(project_desc, project_tech, keywords):
    """Enhance project description to highlight relevant skills"""
    prompt = f"""Rewrite this project description to be more professional and highlight technical achievements.

Project: {project_desc}
Technologies: {project_tech}

Enhanced description:"""

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=100,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    improved = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # If output is too short or generic, return original
    if len(improved) < 15:
        return project_desc
    return improved.strip()


def generate_professional_summary(user_data, keywords, job_description):
    """Generate a professional summary tailored to the job"""
    # Get actual experience details
    exp_count = len(user_data.get('experience', []))
    skills = user_data.get('skills', [])[:6]

    # Get first job title if available
    first_role = "Professional"
    if user_data.get('experience') and len(user_data['experience']) > 0:
        first_role = user_data['experience'][0].get('title', 'Professional')

    prompt = f"""Write a professional resume summary for someone with these qualifications:
Role: {first_role}
Years of experience: {exp_count} positions
Skills: {', '.join(skills)}

Write a 2-3 sentence professional summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=120,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True,
        length_penalty=1.0
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Fallback if AI generates poor summary
    if len(summary) < 30 or "job requirements" in summary.lower():
        summary = f"Results-driven {first_role} with experience in {', '.join(skills[:3])}. Proven track record of delivering high-quality solutions and collaborating with cross-functional teams."

    return summary.strip()

In [3]:
def create_professional_resume_pdf(resume_data):
    """Generate a professional, ATS-friendly PDF resume"""
    buffer = io.BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter,
                           rightMargin=0.75*inch, leftMargin=0.75*inch,
                           topMargin=0.75*inch, bottomMargin=0.75*inch)

    story = []
    styles = getSampleStyleSheet()


    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        textColor=colors.HexColor('#2C3E50'),
        spaceAfter=6,
        alignment=TA_CENTER,
        fontName='Helvetica-Bold'
    )

    contact_style = ParagraphStyle(
        'Contact',
        parent=styles['Normal'],
        fontSize=10,
        textColor=colors.HexColor('#34495E'),
        alignment=TA_CENTER,
        spaceAfter=20
    )

    section_header_style = ParagraphStyle(
        'SectionHeader',
        parent=styles['Heading2'],
        fontSize=14,
        textColor=colors.HexColor('#2C3E50'),
        spaceAfter=12,
        spaceBefore=12,
        fontName='Helvetica-Bold',
        borderWidth=1,
        borderColor=colors.HexColor('#3498DB'),
        borderPadding=5,
        backColor=colors.HexColor('#ECF0F1')
    )

    body_style = ParagraphStyle(
        'CustomBody',
        parent=styles['Normal'],
        fontSize=10,
        textColor=colors.HexColor('#2C3E50'),
        spaceAfter=6,
        leading=14
    )

    # Personal Information
    personal = resume_data.get('personal_info', {})
    story.append(Paragraph(personal.get('name', 'Your Name'), title_style))

    contact_info = f"{personal.get('email', '')} | {personal.get('phone', '')} | {personal.get('location', '')}"
    if personal.get('linkedin'):
        contact_info += f" | LinkedIn: {personal.get('linkedin')}"
    story.append(Paragraph(contact_info, contact_style))

    # Professional Summary
    if resume_data.get('professional_summary'):
        story.append(Paragraph("PROFESSIONAL SUMMARY", section_header_style))
        story.append(Paragraph(resume_data['professional_summary'], body_style))
        story.append(Spacer(1, 0.2*inch))

    # Skills
    if resume_data.get('skills'):
        story.append(Paragraph("SKILLS", section_header_style))
        skills_text = " • ".join(resume_data['skills'])
        story.append(Paragraph(skills_text, body_style))
        story.append(Spacer(1, 0.2*inch))

    # Experience
    if resume_data.get('experience'):
        story.append(Paragraph("PROFESSIONAL EXPERIENCE", section_header_style))
        for exp in resume_data['experience']:
            job_title = f"<b>{exp.get('title', 'Position')}</b> | {exp.get('company', 'Company')}"
            story.append(Paragraph(job_title, body_style))

            duration = f"<i>{exp.get('start_date', '')} - {exp.get('end_date', 'Present')} | {exp.get('location', '')}</i>"
            story.append(Paragraph(duration, body_style))
            story.append(Spacer(1, 0.1*inch))

            for bullet in exp.get('bullets', []):
                bullet_text = f"• {bullet}"
                story.append(Paragraph(bullet_text, body_style))

            story.append(Spacer(1, 0.15*inch))

    # Projects
    if resume_data.get('projects'):
        story.append(Paragraph("PROJECTS", section_header_style))
        for project in resume_data['projects']:
            project_title = f"<b>{project.get('name', 'Project')}</b>"
            if project.get('technologies'):
                project_title += f" | <i>{project.get('technologies')}</i>"
            story.append(Paragraph(project_title, body_style))

            if project.get('description'):
                story.append(Paragraph(f"• {project['description']}", body_style))

            story.append(Spacer(1, 0.1*inch))

    # Education
    if resume_data.get('education'):
        story.append(Paragraph("EDUCATION", section_header_style))
        for edu in resume_data['education']:
            edu_text = f"<b>{edu.get('degree', 'Degree')}</b> | {edu.get('institution', 'Institution')}"
            story.append(Paragraph(edu_text, body_style))

            edu_details = f"<i>{edu.get('graduation_date', '')} | GPA: {edu.get('gpa', 'N/A')}</i>"
            story.append(Paragraph(edu_details, body_style))
            story.append(Spacer(1, 0.1*inch))

    # Build PDF
    doc.build(story)
    buffer.seek(0)
    return buffer

In [6]:
from flask import Flask, request, jsonify, send_file

app = Flask(__name__)

@app.route("/")
def home():
    return """
    <h1>Enhanced Resume Generator API with PDF Export</h1>
    <p>API is running! Generate professional resumes with AI.</p>
    <h3>Endpoints:</h3>
    <ul>
        <li><b>POST /api/analyze-job</b> - Extract keywords from job description</li>
        <li><b>POST /api/generate-resume</b> - Generate complete tailored resume (JSON)</li>
        <li><b>POST /api/generate-resume-pdf</b> - Generate and download PDF resume</li>
    </ul>
    <p>Model: FLAN-T5-Large (Improved Quality)</p>
    """


@app.route("/api/analyze-job", methods=["POST"])
def analyze_job():
    """Extract keywords from job description"""
    try:
        data = request.json
        job_description = data.get("job_description", "")

        if not job_description:
            return jsonify({"error": "job_description is required"}), 400

        keywords = extract_keywords_from_job(job_description)

        return jsonify({
            "success": True,
            "keywords": keywords,
            "keywords_list": [k.strip() for k in keywords.split(",") if k.strip()]
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route("/api/generate-resume", methods=["POST"])
def generate_resume():
    """Generate a complete tailored resume (JSON response)"""
    try:
        data = request.json

        job_description = data.get("job_description", "")
        personal_info = data.get("personal_info", {})
        education = data.get("education", [])
        experience = data.get("experience", [])
        projects = data.get("projects", [])
        skills = data.get("skills", [])

        if not job_description:
            return jsonify({"error": "job_description is required"}), 400

        keywords = extract_keywords_from_job(job_description)
        keywords_list = [k.strip() for k in keywords.split(",") if k.strip()]

        # Generate professional summary
        user_background = {
            "experience": experience,
            "skills": skills
        }
        professional_summary = generate_professional_summary(user_background, keywords, job_description)

        # Tailor experience bullets
        tailored_experience = []
        for exp in experience:
            tailored_bullets = []
            for bullet in exp.get("bullets", []):
                # Only tailor if bullet is substantial
                if len(bullet) > 10:
                    improved = tailor_resume_experience(bullet, keywords)
                    tailored_bullets.append(improved)
                else:
                    tailored_bullets.append(bullet)

            tailored_experience.append({
                **exp,
                "bullets": tailored_bullets
            })

        # Tailor project descriptions
        tailored_projects = []
        for project in projects:
            tailored_desc = project.get("description", "")
            if len(tailored_desc) > 15:
                tailored_desc = tailor_project_description(
                    project.get("description", ""),
                    project.get("technologies", ""),
                    keywords
                )

            tailored_projects.append({
                **project,
                "description": tailored_desc
            })

        # Match score
        match_score = 0
        if keywords_list and skills:
            matched = sum(1 for k in keywords_list if any(k.lower() in s.lower() for s in skills))
            match_score = (matched / len(keywords_list)) * 100

        resume_data = {
            "success": True,
            "keywords_extracted": keywords_list,
            "professional_summary": professional_summary,
            "personal_info": personal_info,
            "education": education,
            "experience": tailored_experience,
            "projects": tailored_projects,
            "skills": skills,
            "match_score": round(match_score, 1)
        }

        return jsonify(resume_data)

    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route("/api/generate-resume-pdf", methods=["POST"])
def generate_resume_pdf():
    """Generate and download professional PDF resume"""
    try:
        data = request.json

        job_description = data.get("job_description", "")
        personal_info = data.get("personal_info", {})
        education = data.get("education", [])
        experience = data.get("experience", [])
        projects = data.get("projects", [])
        skills = data.get("skills", [])

        if not job_description:
            return jsonify({"error": "job_description is required"}), 400

        # keyword extraction
        keywords = extract_keywords_from_job(job_description)
        user_background = {"experience": experience, "skills": skills}
        professional_summary = generate_professional_summary(user_background, keywords, job_description)

        # detailing more about prof exp
        tailored_experience = []
        for exp in experience:
            tailored_bullets = []
            for bullet in exp.get("bullets", []):
                if len(bullet) > 10:
                    improved = tailor_resume_experience(bullet, keywords)
                    tailored_bullets.append(improved)
                else:
                    tailored_bullets.append(bullet)
            tailored_experience.append({**exp, "bullets": tailored_bullets})

        # project descriptions boasting them
        tailored_projects = []
        for project in projects:
            tailored_desc = project.get("description", "")
            if len(tailored_desc) > 15:
                tailored_desc = tailor_project_description(
                    project.get("description", ""),
                    project.get("technologies", ""),
                    keywords
                )
            tailored_projects.append({**project, "description": tailored_desc})


        resume_data = {
            "professional_summary": professional_summary,
            "personal_info": personal_info,
            "education": education,
            "experience": tailored_experience,
            "projects": tailored_projects,
            "skills": skills
        }

        # Generate PDF
        pdf_buffer = create_professional_resume_pdf(resume_data)

        filename = f"Resume_{personal_info.get('name', 'User').replace(' ', '_')}_{datetime.now().strftime('%Y%m%d')}.pdf"

        return send_file(
            pdf_buffer,
            mimetype='application/pdf',
            as_attachment=True,
            download_name=filename
        )

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [9]:
from pyngrok import ngrok
import threading

ngrok.set_auth_token("34RuNCVR0o60Xl34SO01IPOJy0p_eu3CHpGA8gjg8sSYc1hR")
port = 5000
public_url = ngrok.connect(port).public_url
print(f"🌐 Public URL: {public_url}")


threading.Thread(target=app.run, kwargs={"port": port}).start()

🌐 Public URL: https://machinable-skyla-consortable.ngrok-free.dev


In [10]:
#code for testing of the api
import requests
import json

api_url = "https://machinable-skyla-consortable.ngrok-free.dev/"

# api testing due extra precaution
print("="*50)
print("Testing /api/analyze-job")
print("="*50)

job_description_data = {
    "job_description": "We are looking for a skilled Software Engineer with experience in Python, Django, and AWS to join our team. Responsibilities include developing web applications and contributing to cloud infrastructure."
}

try:
    response = requests.post(f"{api_url}/api/analyze-job", json=job_description_data)
    response.raise_for_status() # Raise an exception for bad status codes
    print("Response Status Code:", response.status_code)
    print("Response JSON:")
    print(json.dumps(response.json(), indent=2))
except requests.exceptions.RequestException as e:
    print(f"Error testing /api/analyze-job: {e}")

print("\n")

# this sections is for resume generation phase
print("="*50)
print("Testing /api/generate-resume (JSON output)")
print("="*50)

resume_input_data = {
    "job_description": "We are looking for a skilled Software Engineer with experience in Python, Django, and AWS to join our team. Responsibilities include developing web applications and contributing to cloud infrastructure.",
    "personal_info": {
        "name": "Jane Doe",
        "email": "jane.doe@example.com",
        "phone": "123-456-7890",
        "location": "San Francisco, CA",
        "linkedin": "linkedin.com/in/janedoe"
    },
    "education": [
        {
            "degree": "Master of Science in Computer Science",
            "institution": "University of California, Berkeley",
            "graduation_date": "May 2022",
            "gpa": "3.9"
        },
        {
            "degree": "Bachelor of Science in Electrical Engineering",
            "institution": "Stanford University",
            "graduation_date": "May 2020",
            "gpa": "3.8"
        }
    ],
    "experience": [
        {
            "title": "Software Engineer",
            "company": "Tech Solutions Inc.",
            "start_date": "June 2022",
            "end_date": "Present",
            "location": "San Francisco, CA",
            "bullets": [
                "Developed and maintained web applications using Python and Django.",
                "Managed cloud infrastructure on AWS.",
                "Collaborated with cross-functional teams."
            ]
        },
         {
            "title": "Intern",
            "company": "Data Analytics Corp.",
            "start_date": "Summer 2021",
            "end_date": "August 2021",
            "location": "San Francisco, CA",
            "bullets": [
                "Assisted in data analysis projects.",
                "Wrote scripts in Python.",
            ]
        }
    ],
    "projects": [
        {
            "name": "E-commerce Platform",
            "technologies": "React, Node.js, MongoDB",
            "description": "Built a full-stack e-commerce platform."
        },
        {
            "name": "Sentiment Analysis Tool",
            "technologies": "Python, TensorFlow",
            "description": "Developed a machine learning model for sentiment analysis."
        }
    ],
    "skills": ["Python", "Django", "AWS", "React", "Node.js", "MongoDB", "TensorFlow", "SQL", "Docker", "Kubernetes"]
}

try:
    response = requests.post(f"{api_url}/api/generate-resume", json=resume_input_data)
    response.raise_for_status() # Raise an exception for bad status codes
    print("Response Status Code:", response.status_code)
    print("Response JSON:")
    print(json.dumps(response.json(), indent=2))
except requests.exceptions.RequestException as e:
    print(f"Error testing /api/generate-resume: {e}")

print("\n")

# --- Test /api/generate-resume-pdf ---
print("="*50)
print("Testing /api/generate-resume-pdf (PDF download - requires manual check)")
print("="*50)

# This endpoint returns a PDF file. We will just check for a successful response.
try:
    response = requests.post(f"{api_url}/api/generate-resume-pdf", json=resume_input_data)
    response.raise_for_status() # Raise an exception for bad status codes
    print("Response Status Code:", response.status_code)
    print("Response Headers (check for Content-Disposition and Content-Type):")
    for header, value in response.headers.items():
        print(f"  {header}: {value}")

    # You can save the PDF content to a file to verify:
    # with open("generated_resume.pdf", "wb") as f:
    #     f.write(response.content)
    # print("\nPDF content received.")

except requests.exceptions.RequestException as e:
    print(f"Error testing /api/generate-resume-pdf: {e}")

print("\nTesting complete.")

Testing /api/analyze-job


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 15:01:35] "POST /api/analyze-job HTTP/1.1" 200 -


Response Status Code: 200
Response JSON:
{
  "keywords": "Engineer, Python, Django, contributing, cloud",
  "keywords_list": [
    "Engineer",
    "Python",
    "Django",
    "contributing",
    "cloud"
  ],
  "success": true
}


Testing /api/generate-resume (JSON output)


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 15:02:12] "POST /api/generate-resume HTTP/1.1" 200 -


Response Status Code: 200
Response JSON:
{
  "education": [
    {
      "degree": "Master of Science in Computer Science",
      "gpa": "3.9",
      "graduation_date": "May 2022",
      "institution": "University of California, Berkeley"
    },
    {
      "degree": "Bachelor of Science in Electrical Engineering",
      "gpa": "3.8",
      "graduation_date": "May 2020",
      "institution": "Stanford University"
    }
  ],
  "experience": [
    {
      "bullets": [
        "Developed and maintained web applications utilizing Python and Django. Improved bullet point to be professional and impactful.",
        "Enhanced managed cloud infrastructure on AWS. Improved bullet point to be professional and impactful.",
        "Enhanced collaborated with cross-functional teams. Improved collaboration speed and efficiency."
      ],
      "company": "Tech Solutions Inc.",
      "end_date": "Present",
      "location": "San Francisco, CA",
      "start_date": "June 2022",
      "title": "Softwar

INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 15:02:54] "POST /api/generate-resume-pdf HTTP/1.1" 500 -


Error testing /api/generate-resume-pdf: 500 Server Error: Internal Server Error for url: https://machinable-skyla-consortable.ngrok-free.dev//api/generate-resume-pdf

Testing complete.
